In [53]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from sklearn.model_selection import train_test_split
from keras import models
from keras import layers
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras import backend as K

W0606 23:51:47.681900 4536038848 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [4]:
#Gather song paths
group_folders = []
for folder in os.listdir('./audio_files'):
    if folder != '.DS_Store':
        group_folders.append(folder)
song_paths = []
for i in range(len(group_folders)):
    for song in os.listdir('./clipped_audio/'+group_folders[i]):
        if song != '.DS_Store':
            song_paths.append('/clipped_audio/'+group_folders[i]+'/'+song)

In [18]:
#Extract features
data = [[] for x in range(28)]
for song in song_paths:
    y, sr = librosa.load(song[1:], mono=True)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    rmse = librosa.feature.rmse(y=y)
    for i in np.arange(20):
        data[i].append(mfcc[i])
    data[20].append(chroma_stft)
    data[21].append(spec_cent)
    data[22].append(spec_bw)
    data[23].append(rolloff)
    data[24].append(zcr)
    data[25].append(rmse)
    data[26].append(song.split('/')[3])
    data[27].append(song.split('/')[2])

In [23]:
#Create dataframe
dct = {
    'mfcc1': [np.mean(x) for x in data[0]],
    'mfcc2': [np.mean(x) for x in data[1]], 
    'mfcc3': [np.mean(x) for x in data[2]], 
    'mfcc4': [np.mean(x) for x in data[3]], 
    'mfcc5': [np.mean(x) for x in data[4]], 
    'mfcc6': [np.mean(x) for x in data[5]],
    'mfcc7': [np.mean(x) for x in data[6]],
    'mfcc8': [np.mean(x) for x in data[7]],
    'mfcc9': [np.mean(x) for x in data[8]],
    'mfcc10': [np.mean(x) for x in data[9]],
    'mfcc11': [np.mean(x) for x in data[10]],
    'mfcc12': [np.mean(x) for x in data[11]],
    'mfcc13': [np.mean(x) for x in data[12]],
    'mfcc14': [np.mean(x) for x in data[13]],
    'mfcc15': [np.mean(x) for x in data[14]],
    'mfcc16': [np.mean(x) for x in data[15]],
    'mfcc17': [np.mean(x) for x in data[16]],
    'mfcc18': [np.mean(x) for x in data[17]],
    'mfcc19': [np.mean(x) for x in data[18]],
    'mfcc20': [np.mean(x) for x in data[19]],
    'chroma_stft': [np.mean(x) for x in data[20]],
    'spec_cent': [np.mean(x) for x in data[21]],
    'spec_b2': [np.mean(x) for x in data[22]],
    'rolloff': [np.mean(x) for x in data[23]],
    'zcr': [np.mean(x) for x in data[24]],
    'rmse': [np.mean(x) for x in data[25]],
    'song_name': data[26],
    'group': data[27]
}
df = pd.DataFrame(data=dct)

In [28]:
#Split data
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=["group", "song_name"]), 
                                                    df["group"], 
                                                    test_size=0.2, random_state=0)

In [106]:
#Encode y into scalar values and scale x
encoder = LabelEncoder()
scaler = StandardScaler()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)
X_train = scaler.fit_transform(np.array(X_train, dtype=float))
X_test = scaler.fit_transform(np.array(X_test, dtype=float))

In [107]:
#Construct model
model = tf.keras.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [108]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [109]:
#Train model
history = model.fit(X_train,
                    y_train,
                    epochs=10,
                    batch_size=32)

Epoch 1/10
240/240 [==============================] - 0s 861us/sample - loss: 2.1499 - acc: 0.2917
Epoch 2/10
240/240 [==============================] - 0s 52us/sample - loss: 1.7488 - acc: 0.5083
Epoch 3/10
240/240 [==============================] - 0s 52us/sample - loss: 1.3832 - acc: 0.5708
Epoch 4/10
240/240 [==============================] - 0s 53us/sample - loss: 1.0527 - acc: 0.7208
Epoch 5/10
240/240 [==============================] - 0s 54us/sample - loss: 0.7841 - acc: 0.8083
Epoch 6/10
240/240 [==============================] - 0s 60us/sample - loss: 0.5930 - acc: 0.8542
Epoch 7/10
240/240 [==============================] - 0s 54us/sample - loss: 0.4892 - acc: 0.8500
Epoch 8/10
240/240 [==============================] - 0s 54us/sample - loss: 0.3828 - acc: 0.9042
Epoch 9/10
240/240 [==============================] - 0s 57us/sample - loss: 0.3155 - acc: 0.9208
Epoch 10/10
240/240 [==============================] - 0s 54us/sample - loss: 0.2621 - acc: 0.9375


In [110]:
test_loss, test_acc = model.evaluate(X_test,y_test)

60/60 [==============================] - 0s 2ms/sample - loss: 0.9296 - acc: 0.7000


In [112]:
print('Test Set Accuracy:', test_acc)

Test Set Accuracy: 0.7
